## Example on how to score data samples for RRHF

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers.utils.import_utils import is_torch_bf16_gpu_available

# reward model id
model_id = "reciprocate/dahoas-gptj-rm-static"

dtype = torch.bfloat16 if is_torch_bf16_gpu_available() else torch.float16
reward_model = AutoModelForSequenceClassification.from_pretrained(model_id,device_map="auto", torch_dtype=dtype)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import pipeline 

clf = pipeline("text-classification", model=reward_model,tokenizer=tokenizer)

Using /home/ubuntu/.cache/torch_extensions/py39_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py39_cu117/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Failed to load CUDA kernels. Mra requires custom CUDA kernels. Please verify that compatible versions of PyTorch and CUDA Toolkit are installed: Error building extension 'cuda_kernel'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[1/1] c++ cuda_kernel.cuda.o cuda_launch.cuda.o torch_extension.o -shared -L/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/lib -lc10 -lc10_cuda -ltorch_cpu -ltorch_cuda -ltorch -ltorch_python -L/opt/conda/envs/pytorch/lib64 -lcudart -o cuda_kernel.so
FAILED: cuda_kernel.so 
c++ cuda_kernel.cuda.o cuda_launch.cuda.o torch_extension.o -shared -L/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/lib -lc10 -lc10_cuda -ltorch_cpu -ltorch_cuda -ltorch -ltorch_python -L/opt/conda/envs/pytorch/lib64 -lcudart -o cuda_kernel.so
/usr/bin/ld: cannot find -lcudart
collect2: error: ld returned 1 exit status
ninja: build stopped: subcommand failed.


load hh dataset 

In [6]:
from typing import Dict
from random import randrange 
from datasets import load_dataset, Dataset

def extract_anthropic_prompt(prompt_and_response):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term = "\n\nAssistant:"
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]
  
def get_hh(split: str,n_samples=None) -> Dataset:
    """Load the Anthropic Helpful-Harmless dataset from Hugging Face and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts should be structured as follows:
      \n\nHuman: <prompt>\n\nAssistant:
    Multiple turns are allowed, but the prompt should always start with \n\nHuman: and end with \n\nAssistant:.
    """
    dataset = load_dataset("Anthropic/hh-rlhf", split=split)
    if n_samples:
        dataset = dataset.select(range(min(len(dataset), n_samples)))

    def split_prompt_and_responses(sample) -> Dict[str, str]:
        prompt = extract_anthropic_prompt(sample["chosen"])
        return {
            "prompt": prompt,
            "chosen": sample["chosen"][len(prompt) :],
            "rejected": sample["rejected"][len(prompt) :],
        }

    return dataset.map(split_prompt_and_responses)


dataset = get_hh("train")



Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-1cfce6f8a62eee84/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-1cfce6f8a62eee84/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-61d9a6b7f9a3b608.arrow


score random example

In [7]:
# get sample
sample = dataset[randrange(len(dataset)-1)]


# reject sample
print("Rejected sample")
print(clf(sample["rejected"])[0]["score"])

# chosen sample
print("Chosen sample")
print(clf(sample["chosen"])[0]["score"])

Rejected sample
0.2146914005279541
Chosen sample
0.9867772459983826


score dataset 

In [8]:
from functools import partial

def score_data_with_rrhf_format(sample,clf):
  """The dataset is converted to a dictionary with the following structure for RRHF:
    {
        'prompt': str,
        'responses': List[str],
        'scores': List[float],
    }
    We are not using batching since the inputs are not equally long and sequential infernece is equally fast
  """
  score_chosen = clf(sample["chosen"],truncation=True)[0]["score"]
  score_rejected = clf(sample["rejected"],truncation=True)[0]["score"]
  return {
      'prompt': sample['prompt'],
      "responses": [sample["chosen"],sample["rejected"]],
      "scores": [score_chosen,score_rejected]
  }
  
ds = dataset.filter(lambda s: len(s["chosen"]) > 0 and len(s["rejected"]) > 0).map(partial(score_data_with_rrhf_format,clf=clf),remove_columns=dataset.features.keys())

ds.push_to_hub("philschmid/hh-rrhf-dahoas-gptj-rm", "train")

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-1cfce6f8a62eee84/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-8d0500446f50105c.arrow


Map:   0%|          | 0/160736 [00:00<?, ? examples/s]